# Получение и обработка данных в Gitlab

Будем использовать модуль [python-gitlab](https://python-gitlab.readthedocs.io/) .

А конкретно,возможности [Pipelines and jobs](https://python-gitlab.readthedocs.io/en/stable/gl_objects/pipelines_and_jobs.html) .

In [1]:
#!pip install python-gitlab
import gitlab

In [2]:
# проще всего поместить в рабочий каталог модуль local_secrets.py, где объявляются некоторые секретные переменные
from  local_secrets import *

In [3]:
gl = gitlab.Gitlab(url=GITLAB_URL, private_token=GITLAB_TOKEN)

In [4]:
gl.auth()

In [5]:
project = gl.projects.get(GITLAB_PROJECT_SLUG)

In [6]:
# Нужно взять N последних запусков по порядку updated_at в этом pipeline
# в данном примере 7 последних запусков
import itertools
last_num = 7
pipelines = project.pipelines.list(order_by='updated_at',sort='desc',iterator=True)
last_pipelines = itertools.islice(pipelines, last_num)

In [7]:
def extract_from_job(pipeline):
    return [
         (j.duration,j.commit['message'].strip())
         for j in pipeline.jobs.list()
         if j.name =='compile'
    ]
datalist = map(lambda x: x[0],map(extract_from_job,last_pipelines))

In [8]:
import csv
with open('saved.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(datalist)    